In [123]:
import pandas as pd
import sqlite3
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import csv

pd.set_option("display.max_columns", None)

In [124]:
# Data Directory
data_dir = "../../Data/Big-Data-Cup-2021"
bucketless_data_dir = '../../Data/bdc/data'
tracking_data_dir = "{}/TrackingData".format(data_dir)
db_path = '/Users/keltim01/Documents/databases/'

#connect to database 
conn = sqlite3.connect(db_path + 'bdc_2022.db')
cursor = conn.cursor()

## Adavancement Powerplay Structure Index by Matt Cane
Explainer on hockeygraph: https://hockey-graphs.com/2017/02/14/measuring-the-importance-of-structure-on-the-power-play/ 

Schritte:  
- [x] recreate powerplay structure index

- [ ] improved version (using mainly event data)

- [ ] dynamic powerplay structure index using tracking data (space creation, voronoi, pitch control)

### recreate powerplay structure index

average distance of each player’s shots from their average shot location.

Player Structure = (Σ Distance of Shot To Player’s Average Shot Location) / (# of Shots For Player)

Team Structure Index = (Σ # of Shots for Player * Player Structure ) / (Σ # of Shots for Each Player)

In [125]:
df_pbp_player_shots = pd.read_sql('''
SELECT player_name, team_name, powerplay_shots, avg_x_coord, avg_y_coord, sum_distance_to_avg_shot_location, sum_distance_to_avg_shot_location / powerplay_shots player_structure_index
FROM
(SELECT player_name, team_name, COUNT(player_name) powerplay_shots, AVG(x_coord) avg_x_coord, AVG(y_coord) avg_y_coord, SUM(distance) sum_distance_to_avg_shot_location
FROM (
    SELECT player_name, team_name, x_coord, y_coord, avg_x_coord, avg_y_coord, SQRT((x_coord - avg_x_coord) * (x_coord - avg_x_coord) + (y_coord - avg_y_coord) * (y_coord - avg_y_coord)) distance FROM play_by_play 
LEFT JOIN (
    SELECT player_name, COUNT(player_name) powerplay_shots, AVG(x_coord) avg_x_coord, AVG(y_coord) avg_y_coord FROM play_by_play WHERE is_shot = 1 AND strength_state > 0 GROUP BY player_name
    )
USING (player_name)
WHERE is_shot = 1 AND strength_state > 0
)
GROUP BY player_name
)
ORDER BY player_structure_index ASC;
''',conn)

In [126]:
df_pbp_shots = pd.read_sql('''
SELECT player_name, x_coord, y_coord, avg_x_coord, avg_y_coord, SQRT((x_coord - avg_x_coord) * (x_coord - avg_x_coord) + (y_coord - avg_y_coord) * (y_coord - avg_y_coord)) distance FROM play_by_play 
LEFT JOIN (
    SELECT player_name, COUNT(player_name) powerplay_shots, AVG(x_coord) avg_x_coord, AVG(y_coord) avg_y_coord FROM play_by_play WHERE is_shot = 1 AND strength_state > 0 GROUP BY player_name
    )
USING (player_name)
WHERE is_shot = 1 AND strength_state > 0;
''',conn)

In [127]:
df_pbp_player_team = pd.read_sql('''
SELECT team_name, SUM(powerplay_shots) team_shots, SUM(powerplay_shots * player_structure_index) / SUM(powerplay_shots) team_structure_index
FROM (
    SELECT player_name, team_name, powerplay_shots, avg_x_coord, avg_y_coord, sum_distance_to_avg_shot_location, sum_distance_to_avg_shot_location / powerplay_shots player_structure_index
    FROM
    (SELECT player_name, team_name, COUNT(player_name) powerplay_shots, AVG(x_coord) avg_x_coord, AVG(y_coord) avg_y_coord, SUM(distance) sum_distance_to_avg_shot_location
    FROM (
        SELECT player_name, team_name, x_coord, y_coord, avg_x_coord, avg_y_coord, SQRT((x_coord - avg_x_coord) * (x_coord - avg_x_coord) + (y_coord - avg_y_coord) * (y_coord - avg_y_coord)) distance FROM play_by_play 
    LEFT JOIN (
        SELECT player_name, COUNT(player_name) powerplay_shots, AVG(x_coord) avg_x_coord, AVG(y_coord) avg_y_coord FROM play_by_play WHERE is_shot = 1 AND strength_state > 0 GROUP BY player_name
        )
    USING (player_name)
    WHERE is_shot = 1 AND strength_state > 0
    )
    GROUP BY player_name
    )
)
GROUP BY team_name
ORDER BY team_structure_index ASC;
''',conn)

In [128]:
df_pbp_shots

,player_name,x_coord,y_coord,avg_x_coord,avg_y_coord,distance
0,Brianne Jenner,181,40,93.000000,36.000000,88.090862
1,Cayla Barnes,66,64,102.285714,40.142857,43.425987
2,Kelly Pannek,44,55,107.000000,61.000000,63.285069
3,Cayla Barnes,68,41,102.285714,40.142857,34.296427
4,Hilary Knight,53,64,95.000000,60.000000,42.190046
...,...,...,...,...,...,...
119,Elisa Holopainen,152,27,129.750000,22.000000,22.804879
120,Minnamari Tuominen,138,42,110.636364,38.181818,27.628737
121,Nelli Laitinen,176,60,114.400000,53.600000,61.931575
122,Jenni Hiirikoski,147,37,110.166667,47.333333,38.255355


In [129]:
'''
SQR((x_coord - avg_x_coord) * (x_coord - avg_x_coord) + (y_coord - avg_y_coord) * (y_coord - avg_y_coord)) distance
'''

'\nSQR((x_coord - avg_x_coord) * (x_coord - avg_x_coord) + (y_coord - avg_y_coord) * (y_coord - avg_y_coord)) distance\n'

In [130]:
df_pbp_shots

,player_name,x_coord,y_coord,avg_x_coord,avg_y_coord,distance
0,Brianne Jenner,181,40,93.000000,36.000000,88.090862
1,Cayla Barnes,66,64,102.285714,40.142857,43.425987
2,Kelly Pannek,44,55,107.000000,61.000000,63.285069
3,Cayla Barnes,68,41,102.285714,40.142857,34.296427
4,Hilary Knight,53,64,95.000000,60.000000,42.190046
...,...,...,...,...,...,...
119,Elisa Holopainen,152,27,129.750000,22.000000,22.804879
120,Minnamari Tuominen,138,42,110.636364,38.181818,27.628737
121,Nelli Laitinen,176,60,114.400000,53.600000,61.931575
122,Jenni Hiirikoski,147,37,110.166667,47.333333,38.255355


In [135]:
df_pbp_player_shots

,player_name,team_name,powerplay_shots,avg_x_coord,avg_y_coord,sum_distance_to_avg_shot_location,player_structure_index
0,Abby Roque,Olympic (Women) - United States,1,27.000000,33.000000,0.000000,0.000000
1,Alexandra Vafina,Olympic (Women) - Olympic Athletes from Russia,1,179.000000,31.000000,0.000000,0.000000
2,Ashton Bell,Olympic (Women) - Canada,1,165.000000,66.000000,0.000000,0.000000
3,Claire Thompson,Olympic (Women) - Canada,1,52.000000,36.000000,0.000000,0.000000
4,Dani Cameranesi,Olympic (Women) - United States,1,185.000000,48.000000,0.000000,0.000000
5,Ella Shelton,Olympic (Women) - Canada,1,144.000000,42.000000,0.000000,0.000000
6,Evelina Raselli,Olympic (Women) - Switzerland,1,33.000000,16.000000,0.000000,0.000000
7,Fanuza Kadirova,Olympic (Women) - Olympic Athletes from Russia,1,160.000000,33.000000,0.000000,0.000000
8,Hannah Brandt,Olympic (Women) - United States,1,168.000000,66.000000,0.000000,0.000000
9,Jamie Lee Rattray,Olympic (Women) - Canada,1,45.000000,47.000000,0.000000,0.000000


In [132]:
df_pbp_player_team

,team_name,team_shots,team_structure_index
0,Olympic (Women) - Olympic Athletes from Russia,19,10.004744
1,Olympic (Women) - Canada,10,21.603144
2,Olympic (Women) - Switzerland,19,34.246243
3,Olympic (Women) - Finland,47,40.917732
4,Olympic (Women) - United States,29,43.867209


In [133]:
df_pbp_pp_test = df_pbp_shots = pd.read_sql('''
SELECT team_name, COUNT(team_name), SUM(is_goal)
FROM play_by_play
WHERE is_shot = 1 AND strength_state > 0
GROUP BY team_name;
''',conn)

In [134]:
df_pbp_pp_test

,team_name,COUNT(team_name),SUM(is_goal)
0,Olympic (Women) - Canada,10,1
1,Olympic (Women) - Finland,47,5
2,Olympic (Women) - Olympic Athletes from Russia,19,0
3,Olympic (Women) - Switzerland,19,2
4,Olympic (Women) - United States,29,2


## Shortcomings of basic powerplay structure index 

## What can be changed with Event Data ?

## What can be changed with Tracking data?

## Does it change the predictability?
- We will see it when 